<a href="https://colab.research.google.com/github/binderclip/code-snippets-pyspark/blob/master/prjs/001_logs_stats_map_reduce_filter_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

初始化环境

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install py4j

     |████████████████████████████████| 204kB 2.8MB/s 


设置环境变量

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

创建 spark session 和 spark context

In [0]:
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

spark context word count demo

In [10]:
# 每一行 log map 出来一个统计数据，再通过 reduce 来把结果的值合并起来
# 先用一种比较挫的方式实现

def get_count_abc_teamplate():
  return {
      "a_0": {
          "b_0": 0,
          "b_1": 0,
          "c_0": 0,
          "c_1": 0,
      },
      "a_1": {
          "b_0": 0,
          "b_1": 0,
          "c_0": 0,
          "c_1": 0,
      },
  }

def count_abc(s):
  d = get_count_abc_teamplate()
  a = 0 if "a=0" in s else 1
  b = 0 if "b=0" in s else 1
  c = 0 if "c=0" in s else 1
  if a:
    if b:
      d["a_1"]["b_1"] = 1
    else:
      d["a_1"]["b_0"] = 1
    if c:
      d["a_1"]["c_1"] = 1
    else:
      d["a_1"]["c_0"] = 1
  else:
    if b:
      d["a_0"]["b_1"] = 1
    else:
      d["a_0"]["b_0"] = 1
    if c:
      d["a_0"]["c_1"] = 1
    else:
      d["a_0"]["c_0"] = 1
  return d

def count_abc_sum(d1, d2):
  d3 = get_count_abc_teamplate()
  d3["a_1"]["b_1"] = d1["a_1"]["b_1"] + d2["a_1"]["b_1"]
  d3["a_1"]["b_0"] = d1["a_1"]["b_0"] + d2["a_1"]["b_0"]
  d3["a_1"]["c_1"] = d1["a_1"]["c_1"] + d2["a_1"]["c_1"]
  d3["a_1"]["c_0"] = d1["a_1"]["c_0"] + d2["a_1"]["c_0"]
  d3["a_0"]["b_1"] = d1["a_0"]["b_1"] + d2["a_0"]["b_1"]
  d3["a_0"]["b_0"] = d1["a_0"]["b_0"] + d2["a_0"]["b_0"]
  d3["a_0"]["c_1"] = d1["a_0"]["c_1"] + d2["a_0"]["c_1"]
  d3["a_0"]["c_0"] = d1["a_0"]["c_0"] + d2["a_0"]["c_0"]
  return d3

logs_rdd = sc.parallelize([
                           "a=1, b=1, c=0",
                           "a=1, b=0, c=0",
                           "a=0, b=0, c=0",
                           ])
counts = logs_rdd.map(lambda s: count_abc(s)) \
    .reduce(lambda d1, d2: count_abc_sum(d1, d2))
print(counts)

{'a_0': {'b_0': 1, 'b_1': 0, 'c_0': 1, 'c_1': 0}, 'a_1': {'b_0': 1, 'b_1': 1, 'c_0': 2, 'c_1': 0}}


In [0]:
# 每一行 log map 出来一个统计数据，再通过 reduce 来把结果的值合并起来
# 先用一种比较挫的方式实现

def get_count_abc_teamplate():
  return {
      "a_0": {
          "b_0": 0,
          "b_1": 0,
          "c_0": 0,
          "c_1": 0,
      },
      "a_1": {
          "b_0": 0,
          "b_1": 0,
          "c_0": 0,
          "c_1": 0,
      },
  }

def count_abc(s):
  d = get_count_abc_teamplate()
  a = 0 if "a=0" in s else 1
  b = 0 if "b=0" in s else 1
  c = 0 if "c=0" in s else 1
  if a:
    if b:
      d["a_1"]["b_1"] = 1
    else:
      d["a_1"]["b_0"] = 1
    if c:
      d["a_1"]["c_1"] = 1
    else:
      d["a_1"]["c_0"] = 1
  else:
    if b:
      d["a_0"]["b_1"] = 1
    else:
      d["a_0"]["b_0"] = 1
    if c:
      d["a_0"]["c_1"] = 1
    else:
      d["a_0"]["c_0"] = 1
  return d

def count_abc_sum(d1, d2):
  d3 = get_count_abc_teamplate()
  d3["a_1"]["b_1"] = d1["a_1"]["b_1"] + d2["a_1"]["b_1"]
  d3["a_1"]["b_0"] = d1["a_1"]["b_0"] + d2["a_1"]["b_0"]
  d3["a_1"]["c_1"] = d1["a_1"]["c_1"] + d2["a_1"]["c_1"]
  d3["a_1"]["c_0"] = d1["a_1"]["c_0"] + d2["a_1"]["c_0"]
  d3["a_0"]["b_1"] = d1["a_0"]["b_1"] + d2["a_0"]["b_1"]
  d3["a_0"]["b_0"] = d1["a_0"]["b_0"] + d2["a_0"]["b_0"]
  d3["a_0"]["c_1"] = d1["a_0"]["c_1"] + d2["a_0"]["c_1"]
  d3["a_0"]["c_0"] = d1["a_0"]["c_0"] + d2["a_0"]["c_0"]
  return d3

logs_rdd = sc.parallelize([
                           "a=1, b=1, c=0",
                           "a=1, b=0, c=0",
                           "a=0, b=0, c=0",
                           ])
counts = logs_rdd.map(lambda s: count_abc(s)) \
    .reduce(lambda d1, d2: count_abc_sum(d1, d2))
print(counts)

In [13]:
# 每一行 log map 先预处理数据，然后分别 filter count，得到不同 case 的数量

def parse_log(s):
  a = 0 if "a=0" in s else 1
  b = 0 if "b=0" in s else 1
  c = 0 if "c=0" in s else 1
  return a, b, c

logs_rdd = sc.parallelize([
                           "a=1, b=1, c=0",
                           "a=1, b=0, c=0",
                           "a=0, b=0, c=0",
                           ])
parsed_logs_rdd = logs_rdd.map(parse_log)
count_a_0_b_0 = parsed_logs_rdd.filter(lambda t: t[0] == 0 and t[1] == 0).count()
count_a_1_c_0 = parsed_logs_rdd.filter(lambda t: t[0] == 1 and t[2] == 0).count()
print("a=0, b=0:", count_a_0_b_0)
print("a=1, c=0:", count_a_1_c_0)

a=0, b=0: 1
a=1, c=0: 2
